In [44]:
pip install requests



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [45]:
# REQUEST PARAMETERS:
# api_key           - Your API Key from account dashboard. (required)
# bbox              - Bounding box (South-West Lat, South-West Long, North-East Lat, North-East Long) (optional)
# hex               - Filtering by ICAO24 Hex address. (optional)
# reg_number        - Filtering by aircraft Registration number. (optional)
# airline_icao      - Filtering by Airline ICAO code. (optional)
# airline_iata      - Filtering by Airline IATA code. (optional)
# flag              - Filtering by Airline Country ISO 2 code from Countries DB. (optional)
# flight_icao       - Filtering by Flight ICAO code-number. (optional)
# flight_iata       - Filtering by Flight IATA code-number. (optional)
# flight_number     - Filtering by Flight number only. (optional)
# dep_icao          - Filtering by departure Airport ICAO code. (optional)
# dep_iata          - Filtering by departure Airport IATA code. (optional)
# arr_icao          - Filtering by arrival Airport ICAO code. (optional)
# arr_iata          - Filtering by arrival Airport IATA code. (optional)
# _fields           - Fields to return (comma separated, e.g.: hex, airline_iata, lat, lng) (optional)
# _view             - View format - object (default) or array (good for browser) (optional)


In [37]:
# RESPONSE PARAMETERS:

# hex                - ICAO24 Hex address.
# reg_number         - Aircraft Registration Number
# flag               - ISO 2 country code from Countries DB. Available in the Free plan.
# lat                - Aircraft Geo-Latitude for now. Available in the Free plan.
# lng                - Aircraft Geo-Longitude for now. Available in the Free plan.
# alt                - Aircraft elevation for now (meters).
# dir                - Aircraft head direction for now. Available in the Free plan.
# speed              - Aircraft horizontal speed (km) for now.
# v_speed            - Aircraft vertical speed (km) for now.
# squawk             - Aircraft squawk signal code.
# airline_icao       - Airline ICAO code.
# airline_iata       - Airline IATA code.
# aircraft_icao      - Aircraft ICAO type. Available in the Free plan.
# flight_icao        - Flight ICAO code-number.
# flight_iata        - Flight IATA code-number.
# flight_number      - Flight number only.
# dep_icao           - Departure Airport ICAO code.
# dep_iata           - Departure Airport IATA code. Available in the Free plan.
# arr_icao           - Arrival Airport ICAO code.
# arr_iata           - Arrival Airport IATA code.
# updated            - UNIX timestamp of last aircraft signal.
# status             - Current flight status - scheduled, en-route, landed.


In [4]:
import requests
import pandas as pd
import configparser
import os

def make_request_and_create_dataframe(api_key, dep_icao, arr_iata, _fields):
    try:
        parameters = {
            'api_key': api_key,
            'dep_icao': dep_icao,
            'arr_iata': arr_iata,
            '_fields': _fields
        }

        url = "https://airlabs.co/api/v9/flights"

        response = requests.get(url, params=parameters)
        response.raise_for_status()  # Lanzar una excepción si la solicitud no es exitosa

        data = response.json()

        selected_columns = ["reg_number", "flag", "lat", "lng", "alt", "speed", "flight_number", "airline_iata", "updated", "status", "dep_iata", "arr_iata"]
        
        df = pd.DataFrame(data["response"], columns=selected_columns)

        return df
    except requests.exceptions.RequestException as e:
        print(f"Error en la solicitud HTTP: {e}")
        return None
    except KeyError as e:
        print(f"Error al procesar la respuesta JSON: {e}")
        return None

# Ejemplo de uso de la función

# Obtenemos la ubicación del directorio actual (donde se encuentra el archivo .ipynb)
current_directory = os.getcwd()
# Combinamos la ubicación actual con la ruta relativa al archivo config.ini
config_file_path = os.path.join(current_directory, 'config', 'config.ini')

# Creamos un objeto ConfigParser y leemos la API key desde el archivo config.ini
config = configparser.ConfigParser()
config.read(config_file_path)

api_key = config['API']['api_key']
# print("API Key:", api_key) probamos que este OK.

dep_icao = 'SAEZ'
arr_iata = 'MAD'
_fields = 'reg_number,flag,lat,lng,alt,speed,flight_number,airline_iata,updated,status,arr_iata,dep_iata'

result_df = make_request_and_create_dataframe(api_key, dep_icao, arr_iata, _fields)

if result_df is not None:
    print(result_df.to_string(index=False))
else:
    print("Se produjo un error durante la solicitud o el procesamiento de datos.")




reg_number flag       lat        lng   alt  speed flight_number airline_iata    updated   status dep_iata arr_iata
    EC-NGM   ES  -6.72908 -33.910466 11277    924            42           UX 1698699302 en-route      EZE      MAD
    LV-GIF   AR -11.00885 -36.782084 12641    905          1134           AR 1698699302 en-route      EZE      MAD
    EC-NCX   ES -13.55000 -35.660000 11879    953          6856           IB 1698699293 en-route      EZE      MAD
